In [16]:
import dlc_bci as bci
import torch
from torch.autograd import Variable
from torch import nn
from torch import optim
from torch import Tensor
from torch.nn import functional as F
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Load the data

In [45]:
train_input , train_target = bci.load ( root = "./ data_bci")
test_input , test_target = bci.load ( root = "./ data_bci " , train = False )

train_target = Variable(train_target)
train_input = Variable(train_input.unsqueeze(dim=2))
test_input = Variable(test_input.unsqueeze(dim=2))
test_target = Variable(test_target)

mu, std = train_input.data.mean(), train_input.data.std() 
train_input.data.sub_(mu).div_(std)
test_input.data.sub_(mu).div_(std)


print('train_input size:' , train_input.size())
print ( 'train_target size:' , train_target.size())
print ('test_input size:' , test_input.size())
print ('test_target size:' , test_target.size())

('train_input size:', torch.Size([316, 28, 1, 50]))
('train_target size:', torch.Size([316]))
('test_input size:', torch.Size([100, 28, 1, 50]))
('test_target size:', torch.Size([100]))


In [46]:
# The function computes the accuracy
def accuracy(output,target):
    return (output.float()==target.float()).float().sum()/len(output)

## Convolutional neural network

In [53]:
dim0_conv1=28
dim1_conv1=32
dim0_conv2=dim1_conv1
dim1_conv2= 40

fc1_dim0 =  1

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d (28, 32, kernel_size = (1,5))    # after conv: 20*32*1*46
        self.avg_pool1 = nn.AvgPool2d(kernel_size = (1,4))      # size -> 1/4 *size
        self.conv2 = nn.Conv2d (32 , 64, kernel_size = (1,5))   
        self.fc1 = nn.Linear (448 , 50)                         # size : 64 ->32 => 448/2
        self.fc2 = nn.Linear (50 , 2)
        
    def forward (self , x):
        x = self.conv1(x)
        #print ('after first conv',x.size())
        x = F.relu(self.avg_pool1(x))
        x = F.relu(self.conv2(x))
        x = x.view (-1, 448)
        x = F.relu ( self.fc1 (x))
        x = self.fc2(x)
                   #x = F.relu(F.max_pool2d(self.conv1(x), kernel_size = 3, stride =3) )
        #x = F.relu (F.max_pool2d ( self.conv2(x), kernel_size =2, stride =2) )
        return x
    
net = Net()

In [56]:
### train the model
epoch = 1000
batch_size = 20
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr = 0.001,momentum = 0.5)
i = 0
for e in range(epoch):
    for b in range(0,train_input.size(0),batch_size):
        input_ = train_input.narrow(0,b,min(batch_size,train_input.size(0)-b))
        target = train_target.narrow(0,b,min(batch_size,train_input.size(0)-b))
        output = net(input_).view(min(batch_size,train_input.size(0)-b),2)
        loss = criterion(output,target)
        net.zero_grad()
        loss.backward()
        optimizer.step()
    if e%100 == 0: # print the loss every 10 epochs
        output = net(test_input).view(-1,2)
        a,predicted_class = output.max(dim=1)
        output_train = net(train_input).view(-1,2)
        a,predicted_class_train = output.max(dim=1)
        
        #predicted_class = 1*(predicted_class.float().mean(dim=1) > 0.5)
        print(e//100,'loss train:',criterion(output_train,train_target).data[0],'loss test:',criterion(output,test_target).data[0],' accuracy test:',accuracy(predicted_class,test_target).data[0])

(0, 'loss train:', 0.036786358803510666, 'loss test:', 1.1606009006500244, ' accuracy test:', 0.800000011920929)
(1, 'loss train:', 0.02580004185438156, 'loss test:', 1.195967674255371, ' accuracy test:', 0.800000011920929)
(2, 'loss train:', 0.011333034373819828, 'loss test:', 1.2445627450942993, ' accuracy test:', 0.7900000214576721)
(3, 'loss train:', 0.005387601908296347, 'loss test:', 1.294737696647644, ' accuracy test:', 0.7799999713897705)
(4, 'loss train:', 0.0035095438361167908, 'loss test:', 1.3306933641433716, ' accuracy test:', 0.7900000214576721)
(5, 'loss train:', 0.0026846681721508503, 'loss test:', 1.3568470478057861, ' accuracy test:', 0.7799999713897705)
(6, 'loss train:', 0.0022211067844182253, 'loss test:', 1.377345323562622, ' accuracy test:', 0.7799999713897705)
(7, 'loss train:', 0.0019191540777683258, 'loss test:', 1.3959357738494873, ' accuracy test:', 0.7799999713897705)
(8, 'loss train:', 0.0016949467826634645, 'loss test:', 1.4131555557250977, ' accuracy tes

In [22]:
Variable(torch.LongTensor(3).random_(2))

Variable containing:
 0
 1
 0
[torch.LongTensor of size 3]